In [1]:
# Cell 1: Imports & Configuration

import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
import tensorflow as tf
from tqdm import tqdm

# --- CONFIGURATION ---
TFLITE_MODEL_PATH = Path("mobile_TEST.tflite")
IMAGE_FOLDER       = Path(r"C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\NMMS PHOTOS OF 10TH APRIL\NMMS PHOTOS OF 10TH APRIL")
OUTPUT_CSV         = Path("tflite_sigmoid.csv")
IMAGE_SIZE         = (256, 256)       # must match model input
THRESHOLD          = 0.85             # recaptured probability threshold
CLASS_NAMES        = ["not_recaptured", "recaptured"]  # adjust if your labels differ


c:\Users\arpitasharma2\AppData\Local\anaconda3\envs\tflite-env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
# Cell 2: Load and initialize the TFLite interpreter

interpreter = tf.lite.Interpreter(model_path=str(TFLITE_MODEL_PATH))
interpreter.allocate_tensors()

# Get I/O tensor details for later
input_details  = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

print("TFLite model loaded. Input shape:", input_details["shape"])

TFLite model loaded. Input shape: [  1   3 256 256]


In [3]:
# Cell 3: Pre-processing & Inference Helpers (handles both NHWC/NCHW)

# Determine expected data format:
in_shape = input_details["shape"]
# if channels dim is second, it's NCHW; if last, NHWC
DATA_FORMAT = "NCHW" if in_shape[1] == 3 else "NHWC"
print("Using", DATA_FORMAT, "data format for TFLite input.")

def preprocess_image(img_path: Path) -> np.ndarray:
    """Load image, resize, normalize, batch-ify, and transpose if needed."""
    img = Image.open(img_path).convert("RGB")
    img = img.resize(IMAGE_SIZE, Image.BILINEAR)
    
    arr = np.array(img, dtype=np.float32) / 255.0
    # ImageNet normalization
    mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
    std  = np.array([0.229, 0.224, 0.225], dtype=np.float32)
    arr = (arr - mean) / std
    
    # add batch dim -> (1, H, W, 3)
    arr = np.expand_dims(arr, axis=0)
    
    if DATA_FORMAT == "NCHW":
        # transpose to (1, 3, H, W)
        arr = np.transpose(arr, (0, 3, 1, 2))
    return arr

def predict_tflite(img_arr: np.ndarray) -> np.ndarray:
    """Run the interpreter and return raw logits."""
    interpreter.set_tensor(input_details["index"], img_arr.astype(input_details["dtype"]))
    interpreter.invoke()
    return interpreter.get_tensor(output_details["index"])

def sigmoid(logits: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-logits))


Using NCHW data format for TFLite input.


In [4]:
# Cell 4: Gather all image files

image_paths = []
for ext in ("*.jpg","*.jpeg","*.png"):
    image_paths.extend(IMAGE_FOLDER.glob(ext))
image_paths = sorted(image_paths)

print(f"Found {len(image_paths)} images in {IMAGE_FOLDER}")


Found 9548 images in C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\NMMS PHOTOS OF 10TH APRIL\NMMS PHOTOS OF 10TH APRIL


In [5]:
# Cell 5: Run predictions and collect results

records = []
for img_path in tqdm(image_paths, desc="Predicting"):
    try:
        arr    = preprocess_image(img_path)
        logits = predict_tflite(arr)
        probs = sigmoid(logits)
        recaptured_prob = probs[0][0]   # Assuming output shape is (1, 1)
                # Shape: (2,)
        
    
        flag       = int(recaptured_prob >= THRESHOLD)
        pred_cls   = CLASS_NAMES[int(np.argmax(probs))]
        
        records.append({
            "Image": str(img_path),
            "Class": pred_cls,
            "Flag": flag,
            "Recaptured_Probability": recaptured_prob
        })
    except Exception as e:
        print(f"Error on {img_path.name}: {e}")


Predicting:   1%|▏         | 136/9548 [00:05<05:11, 30.23it/s]

Error on 102_1726002003_WC_22012035247758_2025-04-10R1042025111619.jpeg: cannot identify image file 'C:\\Users\\arpitasharma2\\OneDrive - Deloitte (O365D)\\Desktop\\DIU\\NMMS PHOTOS OF 10TH APRIL\\NMMS PHOTOS OF 10TH APRIL\\102_1726002003_WC_22012035247758_2025-04-10R1042025111619.jpeg'


Predicting:   5%|▍         | 473/9548 [00:14<04:53, 30.91it/s]

Error on 1099_1701005060_FP_22012034636038_2025-04-10R1042025101111.jpeg: cannot identify image file 'C:\\Users\\arpitasharma2\\OneDrive - Deloitte (O365D)\\Desktop\\DIU\\NMMS PHOTOS OF 10TH APRIL\\NMMS PHOTOS OF 10TH APRIL\\1099_1701005060_FP_22012034636038_2025-04-10R1042025101111.jpeg'


Predicting:   6%|▌         | 553/9548 [00:17<04:08, 36.13it/s]

...

Predicting:  91%|█████████ | 8655/9548 [05:46<00:19, 46.08it/s]

Error on 809_1717005027_WC_22012035241477_2025-04-10R1042025060047.jpeg: cannot identify image file 'C:\\Users\\arpitasharma2\\OneDrive - Deloitte (O365D)\\Desktop\\DIU\\NMMS PHOTOS OF 10TH APRIL\\NMMS PHOTOS OF 10TH APRIL\\809_1717005027_WC_22012035241477_2025-04-10R1042025060047.jpeg'


Predicting:  96%|█████████▌| 9137/9548 [05:58<00:09, 43.94it/s]

Error on 914_1715001050_AV_22012034546137_2025-04-10R1042025061253.jpeg: cannot identify image file 'C:\\Users\\arpitasharma2\\OneDrive - Deloitte (O365D)\\Desktop\\DIU\\NMMS PHOTOS OF 10TH APRIL\\NMMS PHOTOS OF 10TH APRIL\\914_1715001050_AV_22012034546137_2025-04-10R1042025061253.jpeg'


Predicting: 100%|██████████| 9548/9548 [06:11<00:00, 25.71it/s]


In [6]:
# Cell 6: Save to CSV and display sample

df = pd.DataFrame(records)
df.to_csv(OUTPUT_CSV, index=False)
print(f"Saved {len(df)} predictions to {OUTPUT_CSV}\n")
display(df.head(10))

Saved 9534 predictions to tflite_sigmoid.csv



,Image,Class,Flag,Recaptured_Probability
0,C:\Users\arpitasharma2\OneDrive - Deloitte (O3...,not_recaptured,1,0.887764
1,C:\Users\arpitasharma2\OneDrive - Deloitte (O3...,not_recaptured,1,0.997907
2,C:\Users\arpitasharma2\OneDrive - Deloitte (O3...,not_recaptured,1,0.988164
3,C:\Users\arpitasharma2\OneDrive - Deloitte (O3...,not_recaptured,0,0.513840
4,C:\Users\arpitasharma2\OneDrive - Deloitte (O3...,not_recaptured,1,0.994476
5,C:\Users\arpitasharma2\OneDrive - Deloitte (O3...,not_recaptured,1,0.893854
6,C:\Users\arpitasharma2\OneDrive - Deloitte (O3...,not_recaptured,1,0.993382
7,C:\Users\arpitasharma2\OneDrive - Deloitte (O3...,not_recaptured,1,0.984687
8,C:\Users\arpitasharma2\OneDrive - Deloitte (O3...,not_recaptured,1,0.997259
9,C:\Users\arpitasharma2\OneDrive - Deloitte (O3...,not_recaptured,1,0.971774


In [7]:
#flag images 
import shutil
import os

# Define output folders
output_dir = Path(r"C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\Classification\flagged_nnms")
flag_0_dir = output_dir / "0"
flag_1_dir = output_dir / "1"

# Create directories if they don't exist
flag_0_dir.mkdir(parents=True, exist_ok=True)
flag_1_dir.mkdir(parents=True, exist_ok=True)

# Copy images to respective folders
for _, row in df.iterrows():
    src = Path(row['Image'])
    dest_dir = flag_1_dir if row['Flag'] == 1 else flag_0_dir
    try:
        shutil.copy(src, dest_dir)
    except Exception as e:
        print(f" Could not copy {src.name} to {dest_dir}: {e}")

print(f"\n Images have been copied into respective folders:\yu8/>\"n- {flag_0_dir}\n- {flag_1_dir}")



 Images have been copied into respective folders:\yu8/>"n- C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\Classification\flagged_nnms\0
- C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\Classification\flagged_nnms\1
